In [1]:
import os
import pandas as pd
from pathlib import Path

In [2]:
PROJECT_ROOT = Path(os.getenv("PROJECT_ROOT"))
DATA_DIR = PROJECT_ROOT / 'data/input/voivodeships/gus'

In [3]:
df_population = pd.read_csv(DATA_DIR / 'LUDN_2425_CREL_20240706142728.csv', sep=';')
df_vehicles = pd.read_csv(DATA_DIR / 'TRAN_1733_CREL_20240706142837.csv', sep=';')
df_roads = pd.read_csv(DATA_DIR / 'TRAN_2655_CREL_20240706142154.csv', sep=';')

In [4]:
density_data = df_population[df_population['Wskaźniki'] == 'ludność na 1 km2']
density_data = density_data[['Rok', 'Nazwa', 'Wartosc']]
density_data = density_data.rename(columns={'Wartosc': 'population_density'})

population_data = df_population[df_population['Wskaźniki'] == 'ludność w tysiącach']
population_data = population_data[['Rok', 'Nazwa', 'Wartosc']]
population_data = population_data.rename(columns={'Wartosc': 'population'})

In [5]:
vehicles_data = df_vehicles[df_vehicles['Rodzaje pojazdów'] == 'pojazdy samochodowe i ciągniki']

vehicles_data = vehicles_data[['Rok', 'Nazwa', 'Wartosc']]
vehicles_data = vehicles_data.rename(columns={'Wartosc': 'vehicles_count'})

In [6]:
roads_data= df_roads[df_roads['Rodzaje dróg'] == 'ogółem']
roads_data = roads_data[['Rok', 'Nazwa', 'Wartosc']]
roads_data = roads_data.rename(columns={'Wartosc': 'roads_length'})

roads_data

,Rok,Nazwa,roads_length
0,2018,DOLNOŚLĄSKIE,"24333,4"
1,2019,DOLNOŚLĄSKIE,"24548,6"
2,2020,DOLNOŚLĄSKIE,"25322,3"
3,2021,DOLNOŚLĄSKIE,"25427,2"
4,2022,DOLNOŚLĄSKIE,"25562,7"
...,...,...,...
225,2018,ZACHODNIOPOMORSKIE,"19849,3"
226,2019,ZACHODNIOPOMORSKIE,"19406,8"
227,2020,ZACHODNIOPOMORSKIE,"20060,4"
228,2021,ZACHODNIOPOMORSKIE,"19604,2"


In [7]:
df_result = density_data \
    .merge(population_data, on=['Nazwa', 'Rok'], how='inner') \
    .merge(vehicles_data, on=['Nazwa', 'Rok'], how='inner') \
    .merge(roads_data, on=['Nazwa', 'Rok'], how='inner') \
    .rename(columns={'Nazwa': 'voivodeship', 'Rok': 'year'}) \
    .assign(voivodeship=lambda x: x['voivodeship'].str.lower()) \
    .replace({',': '.'}, regex=True) \
    .astype({'population_density': float, 'population': float, 'vehicles_count':int, 'roads_length': float}) \
    .sort_values(['voivodeship', 'year']) \
    .reset_index(drop=True)

df_result

,year,voivodeship,population_density,population,vehicles_count,roads_length
0,2018,dolnośląskie,145.4,2901.23,2274325,24333.4
1,2019,dolnośląskie,145.4,2900.16,2366425,24548.6
2,2020,dolnośląskie,145.8,2908.38,2448105,25322.3
3,2021,dolnośląskie,145.3,2897.74,2529725,25427.2
4,2022,dolnośląskie,144.8,2888.03,2592868,25562.7
...,...,...,...,...,...,...
75,2018,świętokrzyskie,106.0,1241.55,1010002,17491.7
76,2019,świętokrzyskie,105.4,1233.96,1048628,17642.8
77,2020,świętokrzyskie,102.4,1199.58,1078563,17499.6
78,2021,świętokrzyskie,101.4,1187.69,1112055,17467.2


In [8]:
df_result.to_csv(PROJECT_ROOT / "data/intermediate/voivodeships.csv", index=False)